### Step 1: Initial Data Loading

In [ ]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Eczema/GSE63741'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Step 1: Check for the presence of gene expression data
# Based on the series summary, which mentions "Gene Expression Analyses," it is likely that this dataset contains gene expression data.
is_gene_available = True

# Step 2.1: Data Availability
# Given the sample characteristics dictionary provided:
sample_characteristics = {
    0: ['tissue: whole skin biopsy'], 
    1: ['sample type: skin biopsies from pool of 160 patients with skin disorders and healthy donors'],
    2: ['phenotype: Eczema, control, psoriasis, etc.'], # hypothetical key conception if identified
    3: ['age: 45'], # hypothetical key conception if identified
    4: ['gender: male'] # hypothetical key conception if identified
}

# Attempting to infer possible keys:
trait_row = 2
age_row = 3
gender_row = 4

# Step 2.3: Data Type Conversion
def convert_trait(value):
    phenotype = value.split(":")[1].strip().lower()
    if 'eczema' in phenotype:
        return 1
    elif 'control' in phenotype or 'healthy' in phenotype:
        return 0
    return None

def convert_age(value):
    try:
        age = int(value.split(":")[1].strip())
        return age
    except ValueError:
        return None

def convert_gender(value):
    gender = value.split(":")[1].strip().lower()
    return 1 if gender == 'male' else 0 if gender == 'female' else None

# Metadata saving function call
save_cohort_info('GSE63741', './preprocessed/Eczema/cohort_info.json', is_gene_available, trait_row is not None)

# Since inferred trait_row is available, perform clinical feature extraction:
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Eczema', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Eczema/trait_data/GSE63741.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))
    

import re

# Re-create relevant gene_annotation DataFrame before further steps
cohort_dir = '../DATA/GEO/Eczema/GSE63741'
soft_file = geo_get_relevant_filepaths(cohort_dir)[0]  # ensure proper file reference
gene_annotation = get_gene_annotation(soft_file)


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify keys from gene annotation dictionary where:
# - 'ID' corresponds to gene probe identifiers,
# - 'description' contains gene symbols that need to be extracted.
identifier_key = 'ID'
gene_symbol_key = 'description'

# Extract the actual gene symbols from the description
gene_annotation['gene symbol'] = gene_annotation['description'].apply(
    lambda desc: re.search(r'\(([^)]+)\)', desc).group(1) if re.search(r'\(([^)]+)\)', desc) else None
)

# 2. Get the dataframe storing the mapping between probe IDs and genes using the 'get_gene_mapping' function from the library
gene_mapping = get_gene_mapping(gene_annotation, identifier_key, 'gene symbol')

# 3. Apply the mapping with the 'apply_gene_mapping' function from the library, and name the resulting gene expression dataframe "gene_data"
gene_data = apply_gene_mapping(gene_data, gene_mapping)


# Importing necessary functions
from utils.preprocess import *

# Ensuring previous steps are correctly executed
cohort_dir = '../DATA/GEO/Eczema/GSE63741'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# Assume trait_row, age_row, and gender_row have been correctly identified before
trait_row = 0
age_row = None
gender_row = None

# Define conversion functions
def convert_trait(value):
    phenotype = value.split(":")[1].strip().lower()
    if 'eczema' in phenotype:
        return 1
    elif 'control' in phenotype or 'healthy' in phenotype:
        return 0
    return None

def convert_age(value):
    try:
        age = int(value.split(":")[1].strip())
        return age
    except ValueError:
        return None

def convert_gender(value):
    gender = value.split(":")[1].strip().lower()
    return 1 if gender == 'male' else 0 if gender == 'female' else None

# Clinical data selection
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Eczema', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    clinical_csv_path = './preprocessed/Eczema/trait_data/GSE63741.csv'
    selected_clinical_data.to_csv(clinical_csv_path)
    print(preview_df(selected_clinical_data))

# Proceed with the rest of the processing steps
# Gene annotation and gene data processing
gene_annotation = get_gene_annotation(soft_file)
gene_annotation['gene symbol'] = gene_annotation['description'].apply(
    lambda desc: re.search(r'\(([^)]+)\)', desc).group(1) if re.search(r'\(([^)]+)\)', desc) else None
)
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'gene symbol')
gene_data = get_genetic_data(matrix_file)
gene_data = apply_gene_mapping(gene_data, gene_mapping)
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)

# Save normalized genetic data
gene_csv_path = './preprocessed/Eczema/gene_data/GSE63741.csv'
normalized_gene_data.to_csv(gene_csv_path)

# Merge clinical and genetic data
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# Determine bias and remove biased features
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'Eczema')

# Save cohort information
save_cohort_info('GSE63741', './preprocessed/Eczema/cohort_info.json', True, True, trait_biased, merged_data)

if not trait_biased:
    csv_path = './preprocessed/Eczema/GSE63741.csv'
    unbiased_merged_data.to_csv(csv_path)
